In [2]:
import os
import sys
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
from IPython.display import Markdown
from itertools import combinations
from scipy.spatial.distance import cdist
from scipy.optimize import linprog
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_absolute_percentage_error, root_mean_squared_error
from sklearn.linear_model import LinearRegression

# Add the parent directory to the Python path to load funtions from file ML_funtions
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)

# Import helperfunctions
from ML_functions import fun_load_data, fun_preprocessing, fun_convert_time, fun_save_file

# Assign string "TSP" or "CVRP" to the following variable to define the optimization problem
optimization_problem = "TSP_blended_proxy"

# Load data
data, start_script = fun_load_data(optimization_problem)

# **Apply the Blended Proxy as defined in Aziz et al., 2016**
### **Add Depot Distance Fraction as feature called Φ DEPOT**

In [37]:
# Get a list of all instance sizes and the number of instances per size
num_customers_list = np.unique(data["Number Customers"])
number_of_instances_per_size = int(len(data[data["Number Customers"] == num_customers_list[0]]) / num_customers_list[0])
print("Instance sizes: {}\nNumber of instances per size: {}".format(num_customers_list, number_of_instances_per_size))

# Add the fraction of the depot distance in the total depot distance of the instance for each customer and multiply it with the total costs
data_set = data.copy()
data_set["Φ DEPOT"] = data_set.groupby("Instance ID").apply(
    lambda group: (group["Depot Distance"] / group["Depot Distance"].sum()) * group["Total Costs"]
    ).reset_index(drop=True)
data_set = data_set[["Instance ID", "Number Customers", "X", "Y", "X Depot", "Y Depot", 
                     "Depot Distance", "Total Costs", "Shapley Value", "SHAPO", "Φ DEPOT"]]
data_set.head()

Instance sizes: [ 6  7  8  9 10 11 12 13 14]
Number of instances per size: 1000


,Instance ID,Number Customers,X,Y,X Depot,Y Depot,Depot Distance,Total Costs,Shapley Value,SHAPO,Φ DEPOT
0,1,6,11.757432,50.848731,2.380844,66.016752,17.832253,227.291186,6.805996,6.847093,12.131437
1,1,6,83.228495,41.537025,2.380844,66.016752,84.472480,227.291186,73.361446,72.126416,57.467361
2,1,6,33.032921,29.876631,2.380844,66.016752,47.388376,227.291186,21.568006,21.751330,32.238724
3,1,6,42.131509,30.755973,2.380844,66.016752,53.136032,227.291186,25.980268,26.403603,36.148904
4,1,6,54.103013,58.267699,2.380844,66.016752,52.299433,227.291186,30.191952,30.346193,35.579758


In [38]:
# Get a list of all instance sizes and the number of instances per size
num_customers_list = np.unique(data["Number Customers"])
number_of_instances_per_size = int(len(data[data["Number Customers"] == num_customers_list[0]]) / num_customers_list[0])
print("Instance sizes: {}\nNumber of instances per size: {}".format(num_customers_list, number_of_instances_per_size))

# Add the fraction of the depot distance to the total depot distance of the instance for each customer
data_set = data.copy()
data_set["Φ DEPOT"] = data_set.groupby("Instance ID")["Depot Distance"].transform(lambda group: group / np.sum(group))
data_set["Φ DEPOT"] = data_set["Φ DEPOT"] * data_set["Total Costs"]
data_set = data_set[["Instance ID", "Number Customers", "X", "Y", "X Depot", "Y Depot", 
                     "Depot Distance", "Total Costs", "Shapley Value", "SHAPO", "Φ DEPOT"]]
data_set.head()

Instance sizes: [ 6  7  8  9 10 11 12 13 14]
Number of instances per size: 1000


,Instance ID,Number Customers,X,Y,X Depot,Y Depot,Depot Distance,Total Costs,Shapley Value,SHAPO,Φ DEPOT
0,1,6,11.757432,50.848731,2.380844,66.016752,17.832253,227.291186,6.805996,6.847093,12.131437
1,1,6,83.228495,41.537025,2.380844,66.016752,84.472480,227.291186,73.361446,72.126416,57.467361
2,1,6,33.032921,29.876631,2.380844,66.016752,47.388376,227.291186,21.568006,21.751330,32.238724
3,1,6,42.131509,30.755973,2.380844,66.016752,53.136032,227.291186,25.980268,26.403603,36.148904
4,1,6,54.103013,58.267699,2.380844,66.016752,52.299433,227.291186,30.191952,30.346193,35.579758


### **Add Nested Moat-Packing as feature called Φ MOAT**

In [3]:
# Define helperfuntions
# Step 4: Check for disconnected components (subtours)
def check_subtours(moat_widths, subsets, n):
    # Construct an adjacency matrix based on the moat widths
    adjacency_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            moat_sum = 0
            for k, S in enumerate(subsets):
                if (i in S and j not in S) or (i not in S and j in S):
                    moat_sum += moat_widths[k]
            adjacency_matrix[i, j] = moat_sum
            adjacency_matrix[j, i] = moat_sum
    
    # Use sparse graph representation to check connected components
    graph = csr_matrix(adjacency_matrix > 0)  # Convert to binary graph (edges exist or not)
    num_components, labels = connected_components(csgraph=graph, directed=False, return_labels=True)
    
    if num_components > 1:
        # Return the subtours (disconnected components)
        subtours = [np.where(labels == i)[0] for i in range(num_components)]
        return subtours
    else:
        return None

# Step 5: Implement moat nesting
def nest_moats(moat_widths, subsets):
    nested_widths = moat_widths.copy()
    
    # Sort the moats by size (smaller subsets first)
    sorted_subsets = sorted(enumerate(subsets), key=lambda x: len(x[1]))
    
    for i, (k, subset) in enumerate(sorted_subsets):
        for j, (k_other, larger_subset) in enumerate(sorted_subsets):
            if subset.issubset(larger_subset) and i != j:
                # Instead of subtracting, we ensure smaller moats are nested inside larger moats
                nested_widths[k_other] = max(nested_widths[k_other], moat_widths[k] + nested_widths[k_other])
    
    return nested_widths

# Step 6: Recalculate the cost allocation
def calculate_cost_allocation(nested_widths, subsets, n):
    allocation = [0] * n
    total_moat_width = sum(nested_widths)
    
    # Allocate costs based on participation in moats
    for i in range(n):
        for k, S in enumerate(subsets):
            if i in S:
                allocation[i] += nested_widths[k] / len(S)
    
    # Normalize allocation to ensure the total cost is the MPV
    total_allocation = sum(allocation)
    if total_allocation > 0:
        allocation = [a * total_moat_width / total_allocation for a in allocation]
    
    return allocation

In [4]:
# Define settings
df_moat = pd.DataFrame()
run_time = {}
prints = True # Show prints and interim results during generating

# Select the instances
min_instance_id = 0 # Set parameter to 0 or 1 to run the algroithm from the first instance
max_instance_id = np.inf # Set parameter to "np.inf" to run the algroithm until the last instance
instances = np.unique(data_set[(data_set["Instance ID"] >= min_instance_id) & (data_set["Instance ID"] <= max_instance_id)]["Instance ID"])

for instance_id in tqdm(instances, desc="Processing", unit="iteration", leave=True):

    if (instance_id % number_of_instances_per_size == 1): start = time.time() # Start time count if a new instance sice starts
    if (max_instance_id > 1): prints = False # No prints for more than one instance

    ###############################################################################
    # Get instance Data Frame, add depot coordinates as row and get distance matrix
    ###############################################################################

    instance_df = data_set[data_set["Instance ID"] == instance_id]
    num_customers = int(instance_df.iloc[0]["Number Customers"])

    # Add depot with X and Y coordinates as first row (like in the original setting from function generate_tsp_instance)
    x_depot = instance_df.iloc[0]["X Depot"] # X coordinate of depot
    y_depot = instance_df.iloc[0]["Y Depot"] # Y coordinate of depot
    df_depot = pd.DataFrame(data={"Instance ID": instance_id, "Number Customers": num_customers, 
                                  "X": x_depot, "Y": y_depot}, index=[0])
    instance_df = pd.concat([df_depot, instance_df], ignore_index=True) # Index starts at zero for all instance_df

    # Calculate distance matrix
    distance_matrix = cdist(instance_df[["X", "Y"]], instance_df[["X", "Y"]], metric="euclidean")
    if (prints == True): display(instance_df, distance_matrix)

    ###############################################################################
    # Apply nested moat packing
    ###############################################################################
    # Define the number of locations (customer plus the depot)
    n = num_customers + 1

    # Step 1: Generate all possible subsets (S, S') of the locations N (except 0)
    subsets = []
    for size in range(1, n):
        subsets += [set(combo) for combo in combinations(range(1, n), size)]

    num_subsets = len(subsets)

    # Objective: maximize the sum of moat widths
    c = [-1] * num_subsets  # Negative because linprog minimizes, we want to maximize

    # Bounds: moat widths must be >= 0
    bounds = [(0, None)] * num_subsets

    # Step 2: Prepare the initial constraints for moat widths
    A = []
    b = []

    for i in range(n):
        for j in range(i + 1, n):
            constraint = [0] * num_subsets
            for k, S in enumerate(subsets):
                if (i in S and j not in S) or (i not in S and j in S):
                    constraint[k] = 1
            A.append(constraint)
            b.append(distance_matrix[i][j])

    # Convert A and b into numpy arrays for compatibility with scipy's linprog
    A = np.array(A)
    b = np.array(b)

    # Step 3: Solve the initial linear program
    res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method="highs")

    # Step 7: Solve the problem and allocate costs

    # Initial solution
    moat_widths = res.x

    # Check for subtours
    subtours = check_subtours(moat_widths, subsets, n)
    while subtours:
        # Add subtour constraints (if any) and re-solve the linear program
        for subtour in subtours:
            new_constraint = [0] * num_subsets
            for k, S in enumerate(subsets):
                if any(i in S for i in subtour) and any(i not in S for i in subtour):
                    new_constraint[k] = 1
            A = np.vstack([A, new_constraint])
            b = np.append(b, 2)  # Subtour constraint
        
        res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method="highs")
        moat_widths = res.x
        subtours = check_subtours(moat_widths)

    # Perform moat nesting
    nested_widths = nest_moats(moat_widths, subsets)

    # Calculate cost allocation based on nested moats
    allocation = calculate_cost_allocation(nested_widths, subsets, n)

    # Scale the predictions such that their sum equals the total costs
    total_costs = instance_df.loc[1, "Total Costs"]
    sum_allocation = np.sum(allocation)
    scaled_allocation = np.array(allocation) * (total_costs / sum_allocation)

    # Add the scaled cost allocation to the Data Frame of the instance
    instance_df["Φ MOAT"] = scaled_allocation

    # Output results
    if (prints == True):
        print("Moat widths:", nested_widths)
        print("Maximum Packing Value (MPV):", sum(nested_widths))
        print("Cost allocation per location:", allocation)
        print("Scaled cost allocation per location:", scaled_allocation)

    ###############################################################################
    # Merge instances and optionally display the final instance Data Frame
    ###############################################################################

    # Drop depot row and reset index
    instance_df.drop(index=0, inplace=True)
    instance_df.reset_index(drop=True)

    # Merge instances
    df_moat = pd.concat([df_moat, instance_df], ignore_index=True)

    # Stop time count for instance size
    if (instance_id % number_of_instances_per_size == 0): run_time[num_customers] = time.time() - start

    # View final instance when there was only one modified instance
    if (max_instance_id == 1): print("\n############### FINAL INSTANCE ###############"), display(instance_df)

# View final DataFrame with merged instances
if (max_instance_id != 1): 
    print("-> Total run time: ", fun_convert_time(seconds=sum(run_time.values())))
    display({"Instance size: {} run time".format(key): fun_convert_time(seconds=value) for key, value in run_time.items()})
    display(df_moat)

# Save file
fun_save_file(data=df_moat, subfolder_path="..\\..\\01_data\\01_TSP", name="tsp_instances_moat.xlsx")
print("File saved succesfully.")

Processing: 100%|██████████| 9000/9000 [8:33:03<00:00,  3.42s/iteration]  

-> Total run time:  8h, 33m


{'Instance size: 6 run time': '4s',
 'Instance size: 7 run time': '7s',
 'Instance size: 8 run time': '15s',
 'Instance size: 9 run time': '51s',
 'Instance size: 10 run time': '2m, 9s',
 'Instance size: 11 run time': '7m, 24s',
 'Instance size: 12 run time': '26m, 0s',
 'Instance size: 13 run time': '1h, 35m',
 'Instance size: 14 run time': '6h, 20m'}

,Instance ID,Number Customers,X,Y,X Depot,Y Depot,Depot Distance,Total Costs,Shapley Value,SHAPO,Φ DEPOT,Φ MOAT
0,1,6,11.757432,50.848731,2.380844,66.016752,17.832253,227.291186,6.805996,6.847093,0.053374,30.438918
1,1,6,83.228495,41.537025,2.380844,66.016752,84.472480,227.291186,73.361446,72.126416,0.252836,53.908965
2,1,6,33.032921,29.876631,2.380844,66.016752,47.388376,227.291186,21.568006,21.751330,0.141839,31.852306
3,1,6,42.131509,30.755973,2.380844,66.016752,53.136032,227.291186,25.980268,26.403603,0.159042,33.513918
4,1,6,54.103013,58.267699,2.380844,66.016752,52.299433,227.291186,30.191952,30.346193,0.156538,31.791732
...,...,...,...,...,...,...,...,...,...,...,...,...
89995,9000,14,35.411268,13.512220,83.308855,83.076767,84.459488,370.421950,32.946966,33.551630,0.105205,26.559562
89996,9000,14,94.027698,4.059342,83.308855,83.076767,79.741125,370.421950,70.014213,69.654750,0.099328,32.775583
89997,9000,14,15.703090,62.406463,83.308855,83.076767,70.695127,370.421950,37.349798,42.540654,0.088060,28.146275
89998,9000,14,46.611090,57.482706,83.308855,83.076767,44.741277,370.421950,16.911138,12.089902,0.055731,26.437231


File saved succesfully.


### **Add Blended Proxy as final prediction called Φ Blend**

In [43]:
# Load df_moat from file if it has been created already
subfolder_path = "..\\..\\01_data\\01_TSP"
file_name = "tsp_instances_moat.xlsx" # Data set includes SHAPO and Φ BLEND
current_directory = os.getcwd()
file_path = os.path.join(current_directory, subfolder_path, file_name)

# Load the file
df_moat = pd.read_excel(io=file_path).drop("Unnamed: 0", axis=1)
df_moat

,Instance ID,Number Customers,X,Y,X Depot,Y Depot,Depot Distance,Total Costs,Shapley Value,SHAPO,Φ DEPOT,Φ MOAT
0,1,6,11.757432,50.848731,2.380844,66.016752,17.832253,227.291186,6.805996,6.847093,12.131437,30.438918
1,1,6,83.228495,41.537025,2.380844,66.016752,84.472480,227.291186,73.361446,72.126416,57.467361,53.908965
2,1,6,33.032921,29.876631,2.380844,66.016752,47.388376,227.291186,21.568006,21.751330,32.238724,31.852306
3,1,6,42.131509,30.755973,2.380844,66.016752,53.136032,227.291186,25.980268,26.403603,36.148904,33.513918
4,1,6,54.103013,58.267699,2.380844,66.016752,52.299433,227.291186,30.191952,30.346193,35.579758,31.791732
...,...,...,...,...,...,...,...,...,...,...,...,...
89995,9000,14,35.411268,13.512220,83.308855,83.076767,84.459488,370.421950,32.946966,33.551630,38.970270,26.559562
89996,9000,14,94.027698,4.059342,83.308855,83.076767,79.741125,370.421950,70.014213,69.654750,36.793181,32.775583
89997,9000,14,15.703090,62.406463,83.308855,83.076767,70.695127,370.421950,37.349798,42.540654,32.619286,28.146275
89998,9000,14,46.611090,57.482706,83.308855,83.076767,44.741277,370.421950,16.911138,12.089902,20.643976,26.437231


In [44]:
# Select the two features for the blended proxy
features = ["Φ DEPOT", "Φ MOAT"] # SHAPO
train_data = df_moat[features + ["Number Customers", "Shapley Value"]]

# Do the train test split during the preprocessing
X_train, X_test, y_train, y_test, train_data = fun_preprocessing(train_data, train_size=0.8)

# Remove the feature "Number Customers" for the blended proxy
for i in [X_train, X_test, train_data]: i.drop("Number Customers", axis=1, inplace=True)
display(Markdown("**Data set for the blended proxy:**"), train_data)

# Train the linear regression model on the training data and make predicitons for the test set
blended_proxy = LinearRegression()
blended_proxy.fit(X_train, y_train)
y_pred = blended_proxy.predict(train_data[features]) # Make predictions for all instances to have the score for the train and test set

# View the final model
coefficients = np.round(blended_proxy.coef_, 2)
bias = np.round(blended_proxy.intercept_, 2)
display(Markdown("**Final model:**")), print(f"Φ BLEND = {coefficients[0]} * {features[0]} + {coefficients[1]} * {features[1]} + {bias}")

# Add the predictions of the blended proxy as column to the train_data Data Frame
df_blend = pd.concat([train_data, pd.DataFrame(data=y_pred, columns=["Φ BLEND"])], axis=1)
display(Markdown("**Data set with predictions of the blended proxy:**"), df_blend)

# Add the predictions of the blended proxy as column to the df_moat and save this Data Frame
df_benchmarks = pd.concat([df_moat, pd.DataFrame(data=y_pred, columns=["Φ BLEND"])], axis=1)
display(Markdown("**Final saved Data Frame with all benchmark predictions:**"), df_benchmarks)

# Save file
fun_save_file(data=df_benchmarks, subfolder_path="..\\..\\01_data\\01_TSP", name="tsp_instances_benchmarks.xlsx")
print("File saved succesfully.")

**Data set for the blended proxy:**

,Φ DEPOT,Φ MOAT,Shapley Value
0,12.131437,30.438918,6.805996
1,57.467361,53.908965,73.361446
2,32.238724,31.852306,21.568006
3,36.148904,33.513918,25.980268
4,35.579758,31.791732,30.191952
...,...,...,...
89995,38.970270,26.559562,32.946966
89996,36.793181,32.775583,70.014213
89997,32.619286,28.146275,37.349798
89998,20.643976,26.437231,16.911138


**Final model:**

Φ BLEND = 0.92 * Φ DEPOT + 0.61 * Φ MOAT + -15.66


**Data set with predictions of the blended proxy:**

,Φ DEPOT,Φ MOAT,Shapley Value,Φ BLEND
0,12.131437,30.438918,6.805996,14.026705
1,57.467361,53.908965,73.361446,69.969918
2,32.238724,31.852306,21.568006,33.359091
3,36.148904,33.513918,25.980268,37.963247
4,35.579758,31.791732,30.191952,36.391426
...,...,...,...,...
89995,38.970270,26.559562,32.946966,36.319218
89996,36.793181,32.775583,70.014213,38.105393
89997,32.619286,28.146275,37.349798,31.451366
89998,20.643976,26.437231,16.911138,19.409313


**Final saved Data Frame with all benchmark predictions:**

,Instance ID,Number Customers,X,Y,X Depot,Y Depot,Depot Distance,Total Costs,Shapley Value,SHAPO,Φ DEPOT,Φ MOAT,Φ BLEND
0,1,6,11.757432,50.848731,2.380844,66.016752,17.832253,227.291186,6.805996,6.847093,12.131437,30.438918,14.026705
1,1,6,83.228495,41.537025,2.380844,66.016752,84.472480,227.291186,73.361446,72.126416,57.467361,53.908965,69.969918
2,1,6,33.032921,29.876631,2.380844,66.016752,47.388376,227.291186,21.568006,21.751330,32.238724,31.852306,33.359091
3,1,6,42.131509,30.755973,2.380844,66.016752,53.136032,227.291186,25.980268,26.403603,36.148904,33.513918,37.963247
4,1,6,54.103013,58.267699,2.380844,66.016752,52.299433,227.291186,30.191952,30.346193,35.579758,31.791732,36.391426
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,9000,14,35.411268,13.512220,83.308855,83.076767,84.459488,370.421950,32.946966,33.551630,38.970270,26.559562,36.319218
89996,9000,14,94.027698,4.059342,83.308855,83.076767,79.741125,370.421950,70.014213,69.654750,36.793181,32.775583,38.105393
89997,9000,14,15.703090,62.406463,83.308855,83.076767,70.695127,370.421950,37.349798,42.540654,32.619286,28.146275,31.451366
89998,9000,14,46.611090,57.482706,83.308855,83.076767,44.741277,370.421950,16.911138,12.089902,20.643976,26.437231,19.409313


File saved succesfully.


In [ ]:
# Compute the errors of the blended proxy (also the R^2 score as in the original paper)
MAPE = np.round(mean_absolute_percentage_error(y_true=df_blend["Shapley Value"], y_pred=y_pred), 4) * 100
RMSE = np.round(root_mean_squared_error(y_true=df_blend["Shapley Value"], y_pred=y_pred), 4)
R2 = np.round(blended_proxy.score(X_test, y_test), 4)
print(f"MAPE: {MAPE} %\nRMSE: {RMSE}\nR2: {R2}")

MAPE: 27.900000000000002 %
RMSE: 9.0873
R2: 0.7644
